In [1]:
import numpy as np
import pandas as pd
from distance_functions_v2 import *
from scipy.stats import pearsonr
from sklearn import manifold
from bokeh.plotting import figure, output_file, show
from bokeh.models import ColumnDataSource, Range1d, LabelSet, Label
from bokeh.io import output_notebook, export_png
from bokeh.resources import CDN
output_notebook(CDN)

Loading BokehJS ...

In [13]:
mydata=pd.read_csv("./Data/fake_data.csv")
my_vals=np.unique(mydata.loc[:,"myvar"])

all_nets={} #definisco il dizionario che contterà tutte le matrici di pearson

In [4]:
mydata.head()

,myid,myvar,g1,g2,g3,g4,g5,g6,g7,g8,g9,g10
1,4744,2,0.138281,0.155625,0.142031,0.104219,0.045312,0.122344,0.076719,0.097969,0.042500,0.075000
2,3243,1,0.049462,0.024346,0.236802,0.057663,0.176832,0.137622,0.205023,0.049718,0.020759,0.041773
3,1663,4,0.023893,0.111574,0.183911,0.178869,0.008110,0.049759,0.006576,0.141166,0.210872,0.085270
4,2244,1,0.127624,0.109274,0.155894,0.089106,0.090924,0.086791,0.117375,0.110762,0.088114,0.024136
5,4838,4,0.052286,0.064693,0.061857,0.166785,0.118220,0.148174,0.057072,0.093407,0.079050,0.158454


In [5]:
print(my_vals)

[1 2 3 4]


In [8]:
#subdata=mydata.loc[mydata["myvar"]==1]
#x = np.array(subdata.iloc[:,3])
#print(x)

In [17]:
for val in my_vals:
    subdata=mydata.loc[mydata["myvar"]==val] #crea una matrice che ha la colonna 'myvar' con lo tesos valore
    n_dim=mydata.shape[1]-2 #togli due perche non vuoi le altr colonne
    mymat=np.zeros((n_dim,n_dim)) #creazione della matrice quadrata 
    for i in range(n_dim-1):  #con questro doppio for riempio la matrice nei posti dove voglio io (senza prendere la diagonale)
        for j in range(i+1,n_dim):
            mymat[i,j]=np.abs(pearsonr(x=np.array(subdata.iloc[:,i+2]), #prendo la colonna (g0)
                                   y=np.array(subdata.iloc[:,j+2]))[0]) #prendo la colonna dopo (g0+1)
                #prendo i due array, corrispondenti alle colonne g0 e g1 e calcolol il coefficiente di pearson
            mymat[j,i]=mymat[i,j]   #lo stesso coefficiente di perason saraà uguale ange per g1 su g0 oltre a g0 su g1
    all_nets["S"+str(val)]=mymat  #lo aggiungo alla mia mista net
#creo una lista di matrici per ogni classe di myvar che ho (4 siccome era 1 2 3 4)

In [18]:
print(all_nets)

{'S1': array([[0.        , 0.05499717, 0.23886978, 0.28392601, 0.10718667,
        0.25015262, 0.26248713, 0.05773529, 0.15498344, 0.02883441],
       [0.05499717, 0.        , 0.23636278, 0.03438882, 0.08867477,
        0.03000684, 0.03934244, 0.19537156, 0.16813869, 0.23701222],
       [0.23886978, 0.23636278, 0.        , 0.31756778, 0.05718126,
        0.10117914, 0.14268496, 0.0069597 , 0.12453947, 0.08529424],
       [0.28392601, 0.03438882, 0.31756778, 0.        , 0.22861851,
        0.08898761, 0.20339207, 0.02991101, 0.16060473, 0.16568857],
       [0.10718667, 0.08867477, 0.05718126, 0.22861851, 0.        ,
        0.2492897 , 0.06300339, 0.38897031, 0.05840487, 0.14686462],
       [0.25015262, 0.03000684, 0.10117914, 0.08898761, 0.2492897 ,
        0.        , 0.05368167, 0.11246324, 0.20922572, 0.04688331],
       [0.26248713, 0.03934244, 0.14268496, 0.20339207, 0.06300339,
        0.05368167, 0.        , 0.39165672, 0.08340259, 0.18517605],
       [0.05773529, 0.19537156, 0.

In [25]:
print(all_nets['S1'].shape)

(10, 10)


In [19]:
mapping={}
for i,j in enumerate(all_nets.keys()):
    mapping[i]=j


In [20]:
mapping

{0: 'S1', 1: 'S2', 2: 'S3', 3: 'S4'}

In [27]:
dist_mat=np.zeros((len(my_vals),len(my_vals))) #preparo una matrice lunga 4x4 che conterra le disteanze
print(dist_mat)

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


In [28]:
for i in range(len(my_vals)-1):
    for j in range(i+1,len(my_vals)):
        dist_mat[i,j]=glocal(all_nets[mapping[i]],all_nets[mapping[j]])[2]
        dist_mat[j,i]=dist_mat[i,j]


In [ ]:
print(dist_mat)

In [ ]:
seed = np.random.RandomState(seed=3)
mds = manifold.MDS(n_components=2, max_iter=3000, eps=1e-9, random_state=seed,
                   dissimilarity="precomputed", n_jobs=1)
pos = mds.fit(dist_mat).embedding_

    



In [1]:
source = ColumnDataSource(data=dict(x=pos[:,0],
                                    y=pos[:,1],
                                    names=list(mapping.values()),
                                    colors=["green","yellow","orange","red"]))


p = figure(plot_width=800, plot_height=800)
p.circle(x='x',y='y', size=10, color='colors', alpha=0.5,source=source)
labels = LabelSet(x='x', y='y', text='names',x_offset=5, y_offset=5, source=source)
p.add_layout(labels)
show(p)

NameError: name 'ColumnDataSource' is not defined